In [1]:
import psycopg2

In [2]:
conString = {'user':"postgres", 'password':"", 'host':"localhost", 'port':"5432", 'database':"mydb"}

In [3]:
def getDBConnection(conString):
    try:
        connection = psycopg2.connect(user = conString['user'], 
                                      password = conString['password'],
                                      host = conString['host'], 
                                      port = conString['port'], 
                                      database = conString['database'])
        
        connection.set_session(autocommit=True)
        
        print('Connection established successfully.')
        return connection
        
    except(Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)
            

def closeConnection(conn):
    #closing database connection.
    if(conn):
        conn.close()
        print("PostgreSQL connection is closed")

In [4]:
def createEmployeeTable(conString):
    command = '''
                CREATE TABLE IF NOT EXISTS Employee(
                    EmployeeID serial PRIMARY KEY,
                    EmployeeName varchar(50) NOT NULL,
                    ManagerID int,
                    FOREIGN KEY (ManagerID) REFERENCES Employee(EmployeeID)
                    ON DELETE SET NULL
                );'''
    conn = getDBConnection(conString)
    cursor = conn.cursor()
    
    try:
        
        cursor.execute(command)
        print('Table Successfully Created.')
        
    except(Exception, psycopg2.Error) as error :
        print ("Error while creating table.", error)
    finally:
        cursor.close()
        closeConnection(conn)
    

In [5]:
def insertEmployeeData(conString:dict, records:list):
    command = 'INSERT INTO Employee (EmployeeName, ManagerID) VALUES (%s, %s)'
    conn = getDBConnection(conString)
    cursor = conn.cursor()
    
    try:
        
        cursor.executemany(command, records)
        print(cursor.rowcount, 'records inserted successfully.')
        
    except(Exception, psycopg2.Error) as error :
        print ("Error while inserting data.", error)
    finally:
        cursor.close()
        closeConnection(conn)   
        

In [6]:
createEmployeeTable(conString)

Connection established successfully.
Table Successfully Created.
PostgreSQL connection is closed


In [7]:
def retrieveEmployees(conString:dict, employeeIDs:tuple = ()):
    conn = getDBConnection(conString)
    
    if employeeIDs:
        command = 'SELECT * FROM Employee WHERE EmployeeID in %s ORDER BY EmployeeID ASC;'
        cursor = conn.cursor()
        try:
            
            cursor.execute(command, (employeeIDs,))
            
            employees_ = cursor.fetchall()
            
            return employees_
        
        except(Exception, psycopg2.Error) as error:
            print ("Unable to retrieve data. ", error)
        finally:
            cursor.close()
            closeConnection(conn)
    else:
        
        command = 'SELECT * FROM Employee ORDER BY EmployeeID ASC;'
        cursor = conn.cursor()
        try:
            cursor.execute(command)
            
            employees_ = cursor.fetchall()
            
            return employees_
        
        except(Exception, psycopg2.Error) as error:
            print ("Unable to retrieve data. ", error)
        finally:
            cursor.close()
            closeConnection(conn)
    

In [8]:
def deleteEmployees(conString:dict, employeeIDs:tuple = ()):
    conn = getDBConnection(conString)
    
    if employeeIDs:
        command = 'DELETE FROM Employee WHERE EmployeeID in (%s);'
        cursor = conn.cursor()
        try:
            
            cursor.execute(command, (employeeIDs, ))
            
            print(cursor.rowcount, 'records removed.')
        
        except(Exception, psycopg2.Error) as error:
            print ("Unable to delete data. ", error)
        finally:
            cursor.close()
            closeConnection(conn)
    else:
        
        command = 'DELETE FROM Employee;'
        cursor = conn.cursor()
        try:
            
            cursor.execute(command)
            
            print(cursor.rowcount, 'records removed.')
            
        except(Exception, psycopg2.Error) as error:
            print ("Unable to delete data. ", error)
        finally:
            cursor.close()
            closeConnection(conn)
    

In [9]:
def updateOrgHierarchy(conString:dict, orgHierarchy:list = []):
    conn = getDBConnection(conString)
    
    command = 'UPDATE Employee SET ManagerID = %s WHERE EmployeeID = %s;'
    cursor = conn.cursor()
    try:
        cursor.executemany(command, orgHierarchy)

        print(cursor.rowcount, 'records updated.')

    except(Exception, psycopg2.Error) as error:
        print ("Unable to update data. ", error)
    finally:
        cursor.close()
        closeConnection(conn)
# orgHierarchy = [(12, 19), (13, 12), (14,17), (15,12), (16,14), (17, 19), (18, 15), (20, 17), (21, 14)] # (manager, emp)
    

In [10]:
deleteEmployees(conString)

Connection established successfully.
2 records removed.
PostgreSQL connection is closed


In [11]:
records = [('Ali', None),('Khan', None)]


insertEmployeeData(conString, records)

Connection established successfully.
2 records inserted successfully.
PostgreSQL connection is closed


In [12]:
retrieveEmployees(conString)

Connection established successfully.
PostgreSQL connection is closed


[(32, 'Ali', None), (33, 'Khan', None)]

In [13]:
updateOrgHierarchy(conString, [(30, 3)])

Connection established successfully.
0 records updated.
PostgreSQL connection is closed


In [14]:
retrieveEmployees(conString)

Connection established successfully.
PostgreSQL connection is closed


[(32, 'Ali', None), (33, 'Khan', None)]